# energie production data – 50 Herz

In [ ]:
import pandas as pd

## get raw data

In [ ]:
def convert_to_int(x_str):
    # print(x_str)
    a = x_str.split(".")
    a.extend(a.pop(-1).split(","))
    if len(a) == 1:
        return int(a[0])
    if len(a) == 2:
        return int(a[0])+float(f"0.{a[-1]}")
    else:
        return int(a[0])*1000+int(a[1])+float(f"0.{a[-1]}")

In [ ]:
# read data
df_net = pd.read_csv(
    "../data/Netzeinspeisung_2019.csv", 
    sep=";",
    skiprows=4
).dropna(axis=1)
# convert data
df_net["MW"] = df_net["MW"].apply(lambda x: x*1000)

In [ ]:
# read data
df_sol = pd.read_csv(
    "../data/Solarenergie_Prognose_2019.csv", 
    sep=";",
    skiprows=4
).dropna(axis=1)
# convert data
df_sol["MW"] = df_sol["MW"].apply(convert_to_int)

In [ ]:
# read data
df_win = pd.read_csv(
    "../data/Windenergie_Prognose_2019.csv", 
    sep=";",
    skiprows=4
).dropna(axis=1)
# convert data
df_win = df_win.drop(["Onshore MW", "Offshore MW"], axis=1)
df_win["MW"] = df_win["MW"].apply(convert_to_int)

## combine data

In [ ]:
df_comb = df_net
df_comb["MW_sol"] = df_sol["MW"]
df_comb["MW_win"] = df_win["MW"]

In [ ]:
df_comb["%"] = 100 * (df_comb["MW_win"] + df_comb["MW_sol"]) / df_comb["MW"]

## convert datetime

In [ ]:
raw_date = df_comb['Datum'] + ' ' + df_comb['Von']
date = pd.to_datetime(raw_date, errors='coerce')
df_comb["datelabel"] = date

## drop date, von and bis columns

In [ ]:
df_comb = df_comb.drop(['Datum', 'Von', 'bis'], axis=1)

### problem: 101%
For some points in time the % seems to be more than 100%, which should not be possible. It happens in 3.125% of the cases. <br>
We couldn't figure out the problem, so we just round it down to `100.0`...

In [ ]:
# display problematic rows
df_comb.where(df_comb["%"] > 100.0).dropna().sample(3)

In [ ]:
# cap values at 100.0%
df_comb["%"] = df_comb["%"].apply(lambda x: 100.0 if x > 100.0 else x)

In [ ]:
df_comb.sample(5)

## export relevant data

In [ ]:
df.to_csv(
    "../data/renewData.csv", 
    sep=";", 
    index=False,
    columns=["datelabel", "%"]
)

## vizualisations

In [ ]:
import plotly
import plotly.graph_objects as go
import plotly.express as px

plotly.offline.init_notebook_mode(connected=True)

In [ ]:
fig = px.bar(df_comb, x='datelabel', y='%')
fig.show()